In [2]:
%pip install llama-index-readers-file
%pip install llama-index-vector-stores-faiss
%pip install llama-index-embeddings-huggingface
%pip install llama-index-vector-stores-chroma
%pip install llama-index-llms-gemini
%pip install -q llama-index google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [143]:
from llama_index.llms.gemini import Gemini
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
import os
from pydantic import BaseModel
from llama_index.core.program import LLMTextCompletionProgram
import pandas as pd
import time

In [144]:
load_dotenv()
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")



In [223]:
prompt_template = """\
While sticking to the given pydantic format and writing NONE where module 4 doesnt exist , use the course information given below to generate course details in the required format
{document}
"""



class SubjectInfo(BaseModel):
    """Data model for SubjectInfo"""
    course_code : str
    subject_name : str
    credits : int
    L_T_P_split : str
    total_contact_hours : int
    module_1_info: str
    module_2_info: str
    module_3_info: str
    module_4_info: str
    studio_lab_work: str
    recommended_learning_resources: str
    



In [230]:
loader = SimpleDirectoryReader(input_dir="./app", required_exts=[".pdf"])
documents = loader.load_data()
llm = Gemini(api_key=GOOGLE_API_KEY,temperature=0.4)


In [231]:
program = LLMTextCompletionProgram.from_defaults(output_cls=SubjectInfo,llm=llm,prompt_template_str=prompt_template)
course_data = []
failed_indexes=[]

In [232]:
for i in range(len(documents)):
    try:
        course_data.append(program(document=documents[i].get_content()))
    except Exception as e:
        print(f"Error occurred: {e}")
        print("Retrying after 2 seconds...")
        time.sleep(2)
        try:
            course_data.append(program(document=documents[i].get_content()))
        except Exception as e:
            print(f"Error occurred again: {e}")
            print("Skipping this document...")
            failed_indexes.append(i)

Error occurred: 1 validation error for SubjectInfo
__root__
  Expecting property name enclosed in double quotes: line 1 column 2 (char 1) [type=value_error.jsondecode, input_value='{{"course_code": "CSET21...ISBN 978-9389552416."}}', input_type=str]
Retrying after 2 seconds...
Error occurred again: 1 validation error for SubjectInfo
__root__
  Expecting property name enclosed in double quotes: line 1 column 2 (char 1) [type=value_error.jsondecode, input_value='{{"course_code": "CSET21...ISBN 978-9389552416."}}', input_type=str]
Skipping this document...
Error occurred: 7 validation errors for SubjectInfo
course_code
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type
L_T_P_split
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type
module_1_info
  Input 

In [193]:
course_data_dict = [item.dict() for item in course_data]

In [227]:
df = pd.DataFrame(course_data_dict)

In [235]:
df

,course_code,subject_name,credits,L_T_P_split,total_contact_hours,module_1_info,module_2_info,module_3_info,module_4_info,studio_lab_work,recommended_learning_resources
0,CSET212,Blockchain Foundations,4,3-0-2,42,"Why Blockchain Technology, Blockchain Bitcoin ...","Blockchain types-Public Blockchain, Private Bl...","Proof of Stake, Difference between PoW vs PoS,...","Remix, Compilation of smart contracts, Deploym...","In studio work, students will learn to impleme...","a) Bettina Warburg, Bill Wanger and Tom Serres..."
1,CSET215,Graphics and Visual Computing,4,3-0-2,42,"Importance of Computer Graphics, Video Display...","Cohen Sutherland clipping, Liang Barky clippin...","Affine and coordinate system transformations, ...","Interpolation, Time-varying data, High-dimensi...",Lab work would be conducted in OpenGL. All the...,"a) Pulasthi Gunawardhana, Computer Graphics: T..."
2,CSET217,Software Development with DevOps,4,3-0-2,42,"Fundamentals of Linux: Linux vs Unix, Linux Fi...","Container: Objective and Concept, Container Vs...","CICD and its Pipeline, Implementation and Prin...","DevOps Automation, The Software Delivery Pipel...",The Studio work includes the practical underst...,"a) Rafał Leszko, Continuous Delivery with Dock..."
3,CSET219,Quantum Computing Foundations,4,3-0-2,42,"Quantum computing, Evolution of Quantum System...","Defining Quantum Circuits, Quantum Teleportati...","The HHL Algorithm, solving linear systems of e...","Quantum Merlin-Arthur Problem, The local Hamil...","In-studio work, students will be setting up th...","a) Jack D. Hidary, Quantum Computing: An Appli..."
4,CSET220,Unmanned Aerial Vehicles,4,2-0-4,28,Why Unmanned Aerial Vehicle (UAV)? Introductio...,Basic UAV Aerodynamics: Wing Configuration and...,Coordinate Frame; Earth-Fixed Coordinate Frame...,UAV Controllability; Flight Control; Position ...,In this course students will start with basic ...,"a) Quan Quan, Introduction to Multicopter Desi..."
5,CSET222,"Microcontrollers, Robotics & Embedded Systems",4,2-0-4,28,"Application of Microcontrollers, Why Embedded ...","Raspberry Pi: Architecture, Raspberry Pi Port ...","Fundamentals of Robotics, Robot Operating Syst...","Simple Publisher and Subscriber, Examining the...",Studio work focuses to develop different kind ...,"a) Wyatt Newman, A Systematic Approach to Lear..."
6,CSET223,Augmented Reality Foundations,4,3-0-2,42,"Industrial applicability of AR, AR Systems Ove...","Head motion prediction, Registration, Calibrat...","Physical locomotion techniques, Target based t...","Tracking for Augmented Reality, Augmented Real...",The laboratory of Augmented reality foundation...,"a) Jesse Glover and Jonathan Linowes, Complete..."
7,CSET224,Cloud Computing,4,3-0-2,42,"Cloud Computing, Adoption of cloud-based IT re...","Microservices, Service-Oriented Architecture, ...","Case Study: Cloud Market analysis, Security an...","Comparative study/analysis of public clouds, E...",Practical experience on global cloud infrastru...,"a) Lizhe Wang, Rajiv Ranjan, Jinjun Chen and B..."
8,CSET225,Intelligent Model Design using AI,4,3-0-2,42,Why Intelligent Model? Intelligent Citizens of...,"User privacy, Data Privacy aware AI, Data Shar...","Aligning Mental and Machine Learning Models, C...","Speed vs Accuracy Trade-off, optimizing models...",Students will gain practical experience with t...,"a) Charu C. Aggarwal, Neural Networks and Deep..."
9,CSET226,Blockchain Engineering,4,3-0-2,42,"Blockchain platforms, distributed ledger techn...","IPFS, Decentralized web, Privacy and encryptio...","Decentralized Autonomous Organizations, DAO Me...","Development with Solidity, Truffle framework, ...",The laboratory work enables students to get pr...,"a) Gaur, Nitin and et al, Hands-On Blockchain ..."


In [238]:
df.to_csv('output.csv', index=False)